# Pydeck + Earth Engine: Lines FeatureCollection

This is an example of using Google Earth Engine to visualize a `FeatureCollection` of lines in [Pydeck](https://pydeck.gl). To install and run this notebook locally, refer to the [Pydeck Earth Engine documentation](https://earthengine-layers.com/docs/developer-guide/pydeck-integration).

To see this example online, view the [JavaScript version][js-example].

[js-example]: https://earthengine-layers.com/examples/noaa-hurricanes

Import required packages:

In [ ]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import ee

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

### NOAA Hurricanes dataset

This example uses the [NOAA Atlantic Hurricane catalog][noaa], a dataset with positions of hurricanes and related attributes from 1851 to 2018. In this example we'll look only at hurricanes in 2017.

[noaa]: https://developers.google.com/earth-engine/datasets/catalog/NOAA_NHC_HURDAT2_atlantic

In [ ]:
# Hurricane tracks and points for 2017.
hurricanes = ee.FeatureCollection('NOAA/NHC/HURDAT2/atlantic')

year = '2017'
points = hurricanes.filter(ee.Filter.date(ee.Date(year).getRange('year')))

# Find all of the hurricane ids.
def get_id(point):
    return ee.Feature(point).get('id')
storm_ids = points.toList(1000).map(get_id).distinct()

# Create a line for each hurricane.
def create_line(storm_id):
    pts = points.filter(ee.Filter.eq('id', ee.String(storm_id)))
    pts = pts.sort('system:time_start')
    line = ee.Geometry.LineString(pts.geometry().coordinates())
    feature = ee.Feature(line)
    return feature.set('id', storm_id)

lines = ee.FeatureCollection(storm_ids.map(create_line))

Now we're ready to create the Pydeck layer. The `EarthEngineLayer` makes this simple. Just pass the Earth Engine object to the class.

In [ ]:
lines_layer = EarthEngineLayer(
    lines,
    {'color': 'red'},
    id="tracks",
)

In [ ]:
points_layer = EarthEngineLayer(
    points,
    {'color': 'black'},
    id="points",
)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [ ]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=[points_layer, lines_layer], 
    initial_view_state=view_state
)
r.show()

## Vector

Let's plot these again, but as a vector dataset, instead of a raster dataset

In [ ]:
lines_layer = EarthEngineLayer(
    lines,
    as_vector=True,
    get_line_color=[255, 0, 0],
    getLineWidth=1000,
    lineWidthMinPixels=3,
    id="tracks",
)
points_layer = EarthEngineLayer(
    points,
    get_fill_color=[0, 0, 0],
    pointRadiusMinPixels=3,
    getRadius=100,
    getLineColor=[255, 255, 255],
    lineWidthMinPixels=0.5,
    as_vector=True,
    stroked=True,
    id="points",
)

In [ ]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=[lines_layer, points_layer], 
    initial_view_state=view_state,
    tooltip=True
)
r.show()